In [10]:
from msrest.exceptions import HttpOperationError
from azureml.core import Workspace, Datastore, Dataset
import os

In [15]:
# config
send_dataset_to_cloud = False

In [12]:
#login to Microsoft account
workspace = Workspace.from_config()

In [13]:
datastore = workspace.get_default_datastore()

# blob_datastore_name=os.getenv("BLOB_DATASTORE_NAME") 
# account_name=os.getenv("BLOB_ACCOUNTNAME") # Storage account name
# container_name=os.getenv("BLOB_CONTAINER") # Name of Azure blob container
# account_key=os.getenv("BLOB_ACCOUNT_KEY") # Storage account key

# try:
#     datastore = Datastore.get(workspace, blob_datastore_name)
#     print("Found Blob Datastore with name: %s" % blob_datastore_name)
# except HttpOperationError:
#     datastore = Datastore.register_azure_blob_container(
#        workspace=workspace,
#        datastore_name=blob_datastore_name,
#        account_name=account_name, # Storage account name
#        container_name=container_name, # Name of Azure blob container
#        account_key=account_key) # Storage account key
#     print("Registered blob datastore with name: %s" % blob_datastore_name)

# blob_data_ref = DataReference(
#    datastore=blob_datastore,
#    data_reference_name="blob_test_data",
#    path_on_datastore="testdata")

In [18]:
from azureml.data.dataset_type_definitions import PromoteHeadersBehavior

if send_dataset_to_cloud:
    datastore.upload(src_dir = 'dataset',target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
# create list
datastore_paths = [
    (datastore, 'train-dataset/tabular/por.csv')
#         (datastore, './student-por.csv')
]

school_dataset = Dataset.Tabular.from_delimited_files(
    path=datastore_paths, separator=',',
    header=PromoteHeadersBehavior.ALL_FILES_HAVE_SAME_HEADERS
)

In [21]:
school_dataset = school_dataset.to_pandas_dataframe()

/home/mkielczykowski/miniconda3/envs/AZURE/lib/python3.8/site-packages/azureml/dataprep/api/_dataframereader.py:171: UserWarning: Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.16.0 --upgrade
  warnings.warn('Please install pyarrow>=0.16.0 for improved performance of to_pandas_dataframe. '


In [23]:
school_dataset.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [27]:
# test if header equals the same data as from azure
x = pd.read_csv('dataset/mat.csv')
x.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [28]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(workspace)

[]

In [29]:
import pandas as pd